In [15]:
from pydantic_ai import Agent
from toyaikit.tools import get_instance_methods

from search_tools import SearchTools

In [16]:
import requests
from minsearch import AppendableIndex

In [17]:
def initialize_index():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    documents = []

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            documents.append(doc) 

    index = AppendableIndex(
        text_fields=["question", "text", "section"],
        keyword_fields=["course"]
    )

    index.fit(documents)
    return index


index = initialize_index()
tools = SearchTools(index)

In [21]:
agent_tools = get_instance_methods(tools)
agent_tools

[<bound method SearchTools.add_entry of <search_tools.SearchTools object at 0x781005da7fb0>>,
 <bound method SearchTools.search of <search_tools.SearchTools object at 0x781005da7fb0>>]

In [18]:
agent_instructions = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

If you want to look up the answer, explain why before making the call. Use as many 
keywords from the user question as possible when making first requests.

Make multiple searches. Try to expand your search by using new keywords based on the results you
get from the search.

At the end, make a clarifying question based on what you presented and ask if there are 
other areas that the user wants to explore.
""".strip()

In [22]:
search_agent = Agent(
    name='search',
    model='openai:gpt-4o-mini',
    instructions=agent_instructions,
    tools=agent_tools,
)